In [8]:
import os
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.datasets.folder import default_loader
from torchvision import transforms
import numpy as np
import random
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
import time
import copy
from tqdm import tqdm
import os
import imageio
from skimage import img_as_float
from bm3d import bm3d

In [9]:
# Path to the directory containing your dataset
train_n_dir = "/home/branislava.jankovic/Desktop/denoising-pneumonia/chest_xray/train/NORMAL"
train_p_dir = "/home/branislava.jankovic/Desktop/denoising-pneumonia/chest_xray/train/PNEUMONIA"
val_n_dir = "/home/branislava.jankovic/Desktop/denoising-pneumonia/chest_xray//val/NORMAL"
val_p_dir = "/home/branislava.jankovic/Desktop/denoising-pneumonia/chest_xray/val/PNEUMONIA"
test_n_dir = "/home/branislava.jankovic/Desktop/denoising-pneumonia/chest_xray/test/NORMAL"
test_p_dir = "/home/branislava.jankovic/Desktop/denoising-pneumonia/chest_xray/test/PNEUMONIA"

In [10]:
# List all files in the dataset directory
train_n_files = [f for f in os.listdir(train_n_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
train_p_files = [f for f in os.listdir(train_p_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

val_n_files = [f for f in os.listdir(val_n_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
val_p_files = [f for f in os.listdir(val_p_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

test_n_files = [f for f in os.listdir(test_n_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
test_p_files = [f for f in os.listdir(test_p_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

In [11]:
# Create a directory to save denoised images
output_train_n_dir = "/home/branislava.jankovic/Desktop/denoising-pneumonia/bm3d_dataset/train/NORMAL"
os.makedirs(output_train_n_dir, exist_ok=True)
output_train_p_dir = "/home/branislava.jankovic/Desktop/denoising-pneumonia/bm3d_dataset/train/PNEUMONIA"
os.makedirs(output_train_p_dir, exist_ok=True)

output_val_n_dir = "/home/branislava.jankovic/Desktop/denoising-pneumonia/bm3d_dataset/val/NORMAL"
os.makedirs(output_val_n_dir, exist_ok=True)
output_val_p_dir = "/home/branislava.jankovic/Desktop/denoising-pneumonia/bm3d_dataset/val/PNEUMONIA"
os.makedirs(output_val_p_dir, exist_ok=True)

output_test_n_dir = "/home/branislava.jankovic/Desktop/denoising-pneumonia/bm3d_dataset/test/NORMAL"
os.makedirs(output_test_n_dir, exist_ok=True)
output_test_p_dir = "/home/branislava.jankovic/Desktop/denoising-pneumonia/bm3d_dataset/test/PNEUMONIA"
os.makedirs(output_test_p_dir, exist_ok=True)

In [12]:
# Specify the noise standard deviation (sigma)
sigma = 0.1

# Loop through each image in the dataset
for image_file in train_n_files:
    # Load the noisy image
    image_path = os.path.join(train_n_dir, image_file)
    noisy_image = imageio.imread(image_path)

    # Convert the image to floating-point representation
    noisy_image_float = img_as_float(noisy_image)

    # Transfer the noisy image to the GPU
    noisy_image_gpu = cp.asarray(noisy_image_float)

    # Apply BM3D denoising using GPU acceleration with cupy
    denoised_image_gpu = bm3d(noisy_image_gpu, sigma)

    # Transfer the denoised image back to the CPU
    denoised_image = cp.asnumpy(denoised_image_gpu)

    # Save the denoised image
    output_path = os.path.join(output_train_n_dir, f"denoised_{image_file}")
    imageio.imwrite(output_path, (denoised_image * 255).astype('uint8'))



'\n# Loop through each image in the dataset\nfor image_file in train_n_files:\n    # Load the noisy image\n    image_path = os.path.join(train_n_dir, image_file)\n    noisy_image = imageio.imread(image_path)\n\n    # Convert the image to floating-point representation\n    noisy_image_float = img_as_float(noisy_image)\n\n    # Transfer the noisy image to the GPU\n    noisy_image_gpu = cp.asarray(noisy_image_float)\n\n    # Apply BM3D denoising using GPU acceleration with cupy\n    denoised_image_gpu = bm3d(noisy_image_gpu, sigma)\n\n    # Transfer the denoised image back to the CPU\n    denoised_image = cp.asnumpy(denoised_image_gpu)\n\n    # Save the denoised image\n    output_path = os.path.join(output_train_n_dir, f"denoised_{image_file}")\n    imageio.imwrite(output_path, (denoised_image * 255).astype(\'uint8\'))\n\n'

In [13]:
#  Loop through each image in TRAIN PNEUMONIA
for image_file in train_p_files:
    # Load the noisy image
    image_path = os.path.join(train_p_dir, image_file)
    noisy_image = imageio.imread(image_path)

    # Convert the image to floating-point representation
    noisy_image_float = img_as_float(noisy_image)

    # Apply BM3D denoising
    denoised_image = bm3d(noisy_image_float, sigma)

    # Save the denoised image
    output_path = os.path.join(output_train_p_dir, f"denoised_{image_file}")
    imageio.imwrite(output_path, (denoised_image * 255).astype('uint8'))



'\n\n#  Loop through each image in TRAIN PNEUMONIA\nfor image_file in train_p_files:\n    # Load the noisy image\n    image_path = os.path.join(train_p_dir, image_file)\n    noisy_image = imageio.imread(image_path)\n\n    # Convert the image to floating-point representation\n    noisy_image_float = img_as_float(noisy_image)\n\n    # Apply BM3D denoising\n    denoised_image = bm3d(noisy_image_float, sigma)\n\n    # Save the denoised image\n    output_path = os.path.join(output_train_p_dir, f"denoised_{image_file}")\n    imageio.imwrite(output_path, (denoised_image * 255).astype(\'uint8\'))\n\n'

In [14]:
# Loop through each image in VAL NORMAL
for image_file in val_n_files:
    # Load the noisy image
    image_path = os.path.join(val_n_dir, image_file)
    noisy_image = imageio.imread(image_path)

    # Convert the image to floating-point representation
    noisy_image_float = img_as_float(noisy_image)

    # Apply BM3D denoising
    denoised_image = bm3d(noisy_image_float, sigma)

    # Save the denoised image
    output_path = os.path.join(output_val_n_dir, f"denoised_{image_file}")
    imageio.imwrite(output_path, (denoised_image * 255).astype('uint8'))

/tmp/ipykernel_1369922/529939286.py:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  noisy_image = imageio.imread(image_path)


In [15]:
# Loop through each image in VAL PNEUMONIA
for image_file in val_p_files:
    # Load the noisy image
    image_path = os.path.join(val_p_dir, image_file)
    noisy_image = imageio.imread(image_path)

    # Convert the image to floating-point representation
    noisy_image_float = img_as_float(noisy_image)

    # Apply BM3D denoising
    denoised_image = bm3d(noisy_image_float, sigma)

    # Save the denoised image
    output_path = os.path.join(output_val_p_dir, f"denoised_{image_file}")
    imageio.imwrite(output_path, (denoised_image * 255).astype('uint8'))

/tmp/ipykernel_1369922/1685601100.py:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  noisy_image = imageio.imread(image_path)


In [16]:
# Loop through each image in TEST NORMAL
for image_file in test_n_files:
    # Load the noisy image
    image_path = os.path.join(test_n_dir, image_file)
    noisy_image = imageio.imread(image_path)

    # Convert the image to floating-point representation
    noisy_image_float = img_as_float(noisy_image)

    # Apply BM3D denoising
    denoised_image = bm3d(noisy_image_float, sigma)

    # Save the denoised image
    output_path = os.path.join(output_test_n_dir, f"denoised_{image_file}")
    imageio.imwrite(output_path, (denoised_image * 255).astype('uint8'))

/tmp/ipykernel_1369922/1669538953.py:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  noisy_image = imageio.imread(image_path)


In [17]:
# Loop through each image in TEST PNEUMONIA
for image_file in test_p_files:
    # Load the noisy image
    image_path = os.path.join(test_p_dir, image_file)
    noisy_image = imageio.imread(image_path)

    # Convert the image to floating-point representation
    noisy_image_float = img_as_float(noisy_image)

    # Apply BM3D denoising
    denoised_image = bm3d(noisy_image_float, sigma)

    # Save the denoised image
    output_path = os.path.join(output_test_p_dir, f"denoised_{image_file}")
    imageio.imwrite(output_path, (denoised_image * 255).astype('uint8'))

/tmp/ipykernel_1369922/1227690207.py:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  noisy_image = imageio.imread(image_path)
